Name: **Sharryl Seto (1005523)**

Class: **Cl03**

# PyTorch Tutorial - Sequence Models

In view of the AI quiz, capstone and other deliverables during week 5, the exercises for this lab is made relatively straightforward. The homework exercises can be found at the end of this tutorial.

## Sentiment detection with BERT

The task tackled here is sentiment detection of IMDB articles. In this notebook, we will utilize the Huggingface suite of libraries (transformers, datasets, evaluate, etc) to build upon it to perform some quick prototyping and evaluation of models. Some of the concepts we will use:
- loading pre-trained models
- fine-tuning on downstream tasks
- streamlining the training loop
- evaluation

In [ ]:
import torch
torch.cuda.is_available()

True

First, we install and import various libraries that are useful for this task, namely:
- datasets: Contains numerous popular datasets of different modalities and sizes, along with useful pre-processing tools (https://github.com/huggingface/datasets)
- transformers: Contains numerous pre-trained model from the huggingface community, along with many useful pipelines for automating the training and evaluation process (https://github.com/huggingface/transformers)
- evaluate: Contains numerous standard and custom evaluation metrics that can be easily used as part of the evaluation pipeline (https://github.com/huggingface/evaluate)

In [ ]:
!pip install -U evaluate
!pip install -U datasets
!pip install -U accelerate
!pip install -U transformers

import numpy as np
import pandas as pd
import evaluate
import accelerate
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


The evaluate library provides many standard evaluation metrics (such as f1, precision, accuracy, etc) for specific tasks. For example, the list of standard metrics for the text classification task can be found at https://huggingface.co/tasks/text-classification. Similarly, there are also many custom-made metrics for specific tasks, which the foillowing code prints out.

In [ ]:
metrics_list = evaluate.list_evaluation_modules()
print(metrics_list)

['lvwerra/test', 'jordyvl/ece', 'angelina-wang/directional_bias_amplification', 'cpllab/syntaxgym', 'lvwerra/bary_score', 'hack/test_metric', 'yzha/ctc_eval', 'codeparrot/apps_metric', 'mfumanelli/geometric_mean', 'daiyizheng/valid', 'erntkn/dice_coefficient', 'mgfrantz/roc_auc_macro', 'Vlasta/pr_auc', 'gorkaartola/metric_for_tp_fp_samples', 'idsedykh/metric', 'idsedykh/codebleu2', 'idsedykh/codebleu', 'idsedykh/megaglue', 'cakiki/ndcg', 'Vertaix/vendiscore', 'GMFTBY/dailydialogevaluate', 'GMFTBY/dailydialog_evaluate', 'jzm-mailchimp/joshs_second_test_metric', 'ola13/precision_at_k', 'yulong-me/yl_metric', 'abidlabs/mean_iou', 'abidlabs/mean_iou2', 'KevinSpaghetti/accuracyk', 'NimaBoscarino/weat', 'ronaldahmed/nwentfaithfulness', 'Viona/infolm', 'kyokote/my_metric2', 'kashif/mape', 'Ochiroo/rouge_mn', 'giulio98/code_eval_outputs', 'leslyarun/fbeta_score', 'giulio98/codebleu', 'anz2/iliauniiccocrevaluation', 'zbeloki/m2', 'xu1998hz/sescore', 'dvitel/codebleu', 'NCSOFT/harim_plus', 'JP-S

We now load in the dataset of interest for this task, which is the IMDB dataset that contains a series of movie reviews and the corresponding sentiment label.

In [ ]:
dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


##Transfer Learning

Transfer learning is technique where we take a model that has learnt to solve one problem, then apply it to solving a different but related problem. In this tutorial, we are applying transfer learning by taking the pre-trained BERT-tiny model and fine-tuning it on this task of sentiment detection on IMDB. Details on the BERT-tiny model can be found at https://huggingface.co/google/bert_uncased_L-2_H-128_A-2.

The following code loads a model checkpoint from huggingface (i.e., BERT-tiny as defined in the model_checkpoint variable), which you can use to further fine-tune on the specific downstream task (i.e., IMDB sentiment detection). Apart from a standard huggingface model checkpoint, you can also use this to load one of your previously trained model, e.g., to continue more training or transfer learn on another task. This code also performs some pre-processing of the dataset in terms of tokenization and padding/truncating the text to a specific sequence that BERT requires.

In [ ]:
model_checkpoint = "google/bert_uncased_L-2_H-128_A-2"
max_len = 512

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def preprocess_function(input_data):
    texts = input_data
    return tokenizer(texts["text"], max_length=max_len, padding="max_length", truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)

config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

AutoModelForSequenceClassification provides a high-level interface/wrapper around specific pre-trained models that you can use for general text classification tasks, including finetuning it based on various configurations. More details can be found at https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification.

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, hidden_dropout_prob=0.1)
model.resize_token_embeddings(len(tokenizer)) # need to resize due to new tokens added

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30522, 128, padding_idx=0)

Here, TrainingArguments allow us to specify various parameters relating to the training of our model, including training epochs, batch sizes, etc. More details can be found at https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.TrainingArguments.

In [ ]:
metric_name = 'f1'
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"./snapshots/{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    fp16=True
)

For computation of our evaluation metric

In [ ]:
metric = evaluate.load(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="micro")

Now, we initialize a Trainer that handles the training loop based on our definition of the model and training parameters as defined in the earlier part of this tutorial.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['train'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Start the actual model training.

In [ ]:
train_log = trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.402200,0.364054,0.848760
2,0.364800,0.335384,0.867640


In [ ]:
trainer.save_model("./models/myFinetunedModel") # for saving your model

Finally, we perform our evaluation on our test set using the fine-tuned model from earlier.

In [ ]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device="cuda:0")
results = classifier(dataset['test']['text'], max_length=max_len, padding="max_length", truncation=True)
dfResults = pd.DataFrame.from_dict(results)
dfResults['label'] = dfResults['label'].str.replace('LABEL_','')
f1 = metric.compute(predictions=dfResults['label'].tolist(), references=dataset['test']['label'], average='micro')
print(f1)

{'f1': 0.85304}


## Homework Exercises
**Due: 1st Mar, 11:59pm**
<br>
<br>
Submit your homework as either: (i) an ipynb file with your results inside; or (ii) a python file and separate pdf discussing your results. Based on what you have done so far in this tutorial, repeat another set of experiments with the following changes:





### Part (a)
(a) There is a potential error in the design of the training and testing/evaluation step in this tutorial. Identify what it is and describe how you will solve this.

My answer:

1. The evaluation dataset is taken from the train dataset, as we can see in the step where the Trainer is initialised. We should use a different dataset for training and evaluation to prevent overfitting and allow better generalisation.

2. The test set is also as large as the training set, with 25000 rows.

Potential solution: Let the train-eval split be 80-20.

In [20]:
train_a = encoded_dataset['train'].train_test_split(test_size=0.2)

In [ ]:
trainer_a = Trainer(
    model,
    args,
    train_dataset=train_a['train'],
    eval_dataset=train_a['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [21]:
train_log_a = trainer_a.train()

Epoch,Training Loss,Validation Loss,F1
1,0.307200,0.351839,0.884600
2,0.344700,0.371158,0.888000


Checkpoint destination directory ./snapshots/bert_uncased_L-2_H-128_A-2-finetuned/checkpoint-5000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [22]:
trainer_a.save_model("./models/part_a") # for saving your model

In [23]:
# pipeline handles tokenisation, don't need to encode, can just put raw data for test
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device="cuda:0")
results = classifier(dataset['test']['text'], max_length=max_len, padding="max_length", truncation=True)
dfResults = pd.DataFrame.from_dict(results)
dfResults['label'] = dfResults['label'].str.replace('LABEL_','')
f1 = metric.compute(predictions=dfResults['label'].tolist(), references=dataset['test']['label'], average='micro')
print(f1)

{'f1': 0.8770799999999999}


### Part (b)

(b) Pick another dataset from the datasets library that is not a binary classification task (see https://huggingface.co/datasets). You can sample a subset from this dataset to reduce compute time (see https://huggingface.co/docs/datasets/en/process).


In [26]:
dataset_b = load_dataset("emo")

Generating train split:   0%|          | 0/30160 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5509 [00:00<?, ? examples/s]

In [27]:
print(dataset_b)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 30160
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5509
    })
})


In [28]:
print(dataset_b['train'][0])

{'text': "don't worry  i'm girl hmm how do i know if you are what's ur name", 'label': 0}


In [29]:
model_checkpoint = "google/bert_uncased_L-2_H-128_A-2"
max_len = 512

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def preprocess_function(input_data):
    texts = input_data
    return tokenizer(texts["text"], max_length=max_len, padding="max_length", truncation=True)

# encode
encoded_dataset_b = dataset_b.map(preprocess_function, batched=True)

Map:   0%|          | 0/30160 [00:00<?, ? examples/s]

Map:   0%|          | 0/5509 [00:00<?, ? examples/s]

In [30]:
train_b = encoded_dataset_b['train'].train_test_split(test_size=0.2)

### Part (c)

(c) Fine-tune the same model but with a dropout of 0.2, over 3 epochs and report on accuracy scores, in addition to F1 scores.

In [31]:
model_hw = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=6, hidden_dropout_prob=0.2)
model_hw.resize_token_embeddings(len(tokenizer)) # need to resize due to new tokens added

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30522, 128, padding_idx=0)

In [73]:
metric_name1 = "f1"
metric_name2 = "accuracy"
model_name = model_checkpoint.split("/")[-1]

In [74]:
args = TrainingArguments(
    f"./snapshots/{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3, # 3 epochs
    weight_decay=0.01,
    #load_best_model_at_end=True,
    #metric_for_best_model=metric_name,
    push_to_hub=False,
    fp16=True
)

In [93]:
metric1 = evaluate.load(metric_name1)
metric2 = evaluate.load(metric_name2)

# from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    # using evaluate (micro = accuracy, hence used weighted)
    f1 = metric1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric2.compute(predictions=predictions, references=labels)["accuracy"]
    return {"f1": f1, "accuracy": accuracy}

    # using sklearn
    # return {"f1": f1_score(labels, predictions, average='micro'),
    #         "accuracy": accuracy_score(labels, predictions)}

In [94]:
trainer_c = Trainer(
    model_hw,
    args,
    train_dataset=train_b['train'],
    eval_dataset=train_b['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [95]:
train_log_c = trainer_c.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.158300,0.947422,0.843069,0.841678
2,0.525100,0.743537,0.858202,0.857593
3,0.512400,0.666584,0.857527,0.856764


Checkpoint destination directory ./snapshots/bert_uncased_L-2_H-128_A-2-finetuned/checkpoint-6032 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./snapshots/bert_uncased_L-2_H-128_A-2-finetuned/checkpoint-9048 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [96]:
trainer_c.save_model("./models/part_c1") # for saving your model

In [97]:
# pipeline handles tokenisation, don't need to encode, can just put raw data for test
classifier = pipeline("text-classification", model=model_hw, tokenizer=tokenizer, device="cuda:0")
results = classifier(dataset_b['test']['text'], max_length=max_len, padding="max_length", truncation=True)
dfResults = pd.DataFrame.from_dict(results)
dfResults['label'] = dfResults['label'].str.replace('LABEL_','')

In [98]:
# f1 scores
f1 = metric1.compute(predictions=dfResults['label'].tolist(), references=dataset_b['test']['label'], average='weighted')
print(f1)

{'f1': 0.8429939779410257}


In [99]:
# accuracy
accuracy = metric2.compute(predictions=dfResults['label'].tolist(), references=dataset_b['test']['label'])
print(accuracy)

{'accuracy': 0.8222907968778362}
